In [ ]:
import sys
from bs4 import BeautifulSoup
import functools
import requests

In [ ]:
!pip3 install beautifulsoup4
!pip3 install requests

In [ ]:
def loadElements(inputFile):
    html = open(inputFile).read()
    bs = BeautifulSoup(html)
    elements = bs.select('#anchor_sound_list > div.sound-list._is > ul > li > div.text.lF_')
    return elements

In [ ]:
elements = []
for i in range(1, 7+1):
    inputFile = 'page{}.html'.format(i)
    es = loadElements(inputFile)
    elements.extend(es)

In [ ]:
def parseElement(x):
    y = next(x.children)
    link = y.attrs['href']
    title = y.text
    item_id = link.split('/')[-1]
    return {'item_id': item_id,'title':title}

In [ ]:
items = [parseElement(x) for x in elements]

In [ ]:
@functools.lru_cache()
def getDownloadLink(item_id):
    url = 'https://www.ximalaya.com/revision/play/v1/audio?id={}&ptype=1'.format(item_id)
    print(url)
    r = requests.get(url, headers = {'User-Agent':'curl/7.54.0', 'Content-Type': 'application/json'})
    print(r.status_code, r.content)
    link = r.json()['data']['src']    
    return link

In [ ]:
getDownloadLink('360821293')

In [ ]:
len(items)

In [ ]:
len(set([x['title'] for x in items]))

In [ ]:
for it in items:
    if 'link' not in it:
        link = getDownloadLink(it['item_id'])
#         print('OK!!!')
        it['link'] = link
    else:
        print(it)

In [ ]:
with open('download.sh', 'w') as fh:
    for it in items:
        s = """
        if [ ! -f "audio/{title}.m4a" ]; then
            wget "{url}" -O temp.m4a
            mv temp.m4a "audio/{title}.m4a"
        fi
        """.format(title = it['title'], url = it['link'])
        fh.write(s)